In [1]:
## Install
!pip install easyocr

In [2]:
## Imports

import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.metrics import roc_auc_score

import re
import string
import nltk
import itertools
import numpy as np
import os
import csv
import io

import torch
import cv2
import easyocr

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# this allows you to apply a progress bar to pandas operations
from tqdm import tqdm
tqdm.pandas()

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# easyocr loading
reader = easyocr.Reader(['en']) 

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [4]:
%cd /home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images

/home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images


In [5]:
# # OCR_images에 있는 리스트들로 컬럼 하나 만들기. 리스트 이름 따기.
# import pandas as pd
# import os
# image_df = pd.DataFrame(os.listdir(), columns=['image'])

In [21]:
%cd /home/studio-lab-user/DS-Projects/OCR

/home/studio-lab-user/DS-Projects/OCR


In [22]:
# To pull in previusly save data
import pandas as pd
data = pd.read_csv("easy_results.csv", encoding='latin')
data.head()

,Unnamed: 0,id,description,image,images,relevant,processed_description,text_score
0,0,918707,Boys Among Us Hoodies Super Cool Print Clothes...,968bf5e6c99211c2ad8ac27fa55e4651b941d137,['968bf5e6c99211c2ad8ac27fa55e4651b941d137'],1,boys among us hoodies super cool print clothes...,0.192416
1,1,918708,Cat And Dog Cardboard Printing Jigsaw Puzzle M...,68ece3fab138feb537eee296f3348f3543e23a1d,"['68ece3fab138feb537eee296f3348f3543e23a1d', '...",0,cat dog cardboard printing jigsaw puzzle manuf...,0.030835
2,2,918709,0098 30 Pcs 21pack Sensory 4 Piece Puzzle Fidg...,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,"['a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf', '...",0,0098 30 pcs 21pack sensory 4 piece puzzle fidg...,0.001173
3,3,918710,Boys Sweatshirt Among us 3D Hoodies Cool Fashi...,a278812a832f24a822e2602b91a7b1f5749926cb,['a278812a832f24a822e2602b91a7b1f5749926cb'],1,boys sweatshirt among us 3d hoodies cool fashi...,0.275841
4,4,918711,20cm Among Us Soft Plush Toys Vocal Butt Toy S...,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,['3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a'],0,20cm among us soft plush toys vocal butt toy s...,0.000861


In [23]:
image_df = data[['image','relevant']]

In [24]:
image_df

,image,relevant
0,968bf5e6c99211c2ad8ac27fa55e4651b941d137,1
1,68ece3fab138feb537eee296f3348f3543e23a1d,0
2,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,0
3,a278812a832f24a822e2602b91a7b1f5749926cb,1
4,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,0
5,9a12a490fcea7313378b1e83425fae86ddc81124,0
6,533a514c8c422f93173db706aeb8a2e87603e575,1
7,7dc026543aaf6863f0bc810acda21e888cec7693,0
8,c24c8919bfeb37dbca1422e373d6f79693efedcd,0
9,c74d92fc4f342df4e9a7bf9db1b77ad5cfeea89c,0


In [25]:
image_list = image_df['image'].tolist()

In [26]:
image_list

['...',
 'empty',
 'cfaccfa730fca29e0df6f2e18b74735951d18428',
 '7643759f30a2445e8084ecf848628b8ae7d81a92',
 'fc06cacb287cdea9a186109e792b1cc3b34c97ab',
 'f44b5b470423248e15cb5a28a1200311af2a9c1b',
 '1818410beb2f12283a6ab72db3b988ae56fd3b3e',
 '2c64ca51b53c48fcaf8630d927dc53a4a74a14bf',
 '777f9539646b03a257753ec475c9d2793b0c86ab',
 '02e84e0140c449cf60e451672f210198594ac620',
 '2a1cb620245773522750665f9c99cbc5ef61aebb',
 '3910618b165634d36452bd8d32a6c792ee728150',
 '4111f60e1607757c6f829638268aefe602f33ba7',
 'e7548d35100cf01b8e9dbd44d58163a01632c446',
 '14f4e5198283fbe1a1ffae046ef359af50baee30',
 '6528ef6a197b0c379b1af5afeb33632c151ed814',
 '9a6791a009f973e38fccd9ed56711a79a95ba6d8',
 'd8fdca3761d5fd3a3f99a8aeaeb907107f6a71a3',
 '1e4262b940e28e48b0a674611114884af56a3afe',
 'd0a8cf4cd898ba8c4bc1435a08b0f7f702860b0f',
 '43a18980d983e7835ce8a9f50ae8953390197f73',
 'a2db2c697881750d70feaa1a90d4f271338b9337',
 '077d6694396e202064fb5b01b42837d2949e4ce7',
 'a8057438d83d4fc141347237c23e2d3fc71

In [34]:
# words and scores
def get_words_and_scores_and_add_to_dataframe(row):
    photo = row["image"]
    if photo == "empty":
        return [[],[]]
    
    img_path = f"/home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images/{photo}.jpg"
    # easyocr readtext
    img_text = reader.readtext(img_path)
    #empty lists
    words = []
    confidences = []
    #appending results into the lists
    for (bbox, text, prob) in img_text:
        words.append(text)
        confidences.append(prob)
    result = [words, confidences]
    return result

In [35]:
%cd /home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images

/home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images


In [ ]:
image_df["words"] = image_df.apply(get_words_and_scores_and_add_to_dataframe, axis=1)
image_df

In [ ]:
import re
import string
def clean_text(row):
    word_list = row["words"][0]
    new_word_list = []
    for word in word_list:
        new_word = word.lower()
        new_word = re.sub("[%s]" % re.escape(string.punctuation), " ", new_word)
        new_word_list.append(new_word)
    return [new_word_list, row["words"][1]]

In [ ]:
image_df["words"] = image_df.apply(clean_text, axis=1)

In [ ]:
## define the target words / phrases

targets = ["among us", "impostor"]

In [ ]:
# helper functions for funal scoring function

def advance(iterator, step):
    next(itertools.islice(iterator, step, step), None)


def tuplewize(iterable, size):
    iterators = itertools.tee(iterable, size)
    for position, iterator in enumerate(iterators):
        advance(iterator, position)
    return zip(*iterators)

In [ ]:
# define function to produce final scores

def get_final_score(row):
    sample_list = row["words"][0]
    score_list = row["words"][1]
    max_score = 0
    
    for target in targets:
        phrase_length = len(target.split())
        for c, s in zip(tuplewize(sample_list, phrase_length), tuplewize(score_list, phrase_length)):
            new_sample = " ".join(c)
            edit_distance = nltk.edit_distance(target, new_sample)
            closeness = (len(target)-edit_distance)/len(target)
            confidence_score = np.mean(s)
            final_score = closeness * confidence_score
            if final_score > max_score:
                max_score = final_score
             
    return max_score

In [ ]:
import itertools
import nltk
import numpy as np

image_df["OCR_score"] = image_df.apply(get_final_score, axis=1)

In [ ]:
image_df = image_df.sort_values(by='OCR_score', ascending=False)
pd.set_option('display.max_rows', None)
image_df

In [ ]:
RA_score = roc_auc_score(image_df["relevant"], image_df["OCR_score"])

print("Rekognition ROC AUC Score =  %.2f" % (RA_score))

In [ ]:
# Dataframe of true positives

tp = image_df.loc[(image_df['relevant'] == 1) & (image_df['OCR_score'] >= 0.5)] 
print(len(tp))

In [ ]:
# Dataframe of true negatives

tn = image_df.loc[(image_df['relevant'] == 0) & (image_df['OCR_score'] < 0.5)] 
print(len(tn))

In [ ]:
# Dataframe of false positives

fp = image_df.loc[(image_df['relevant'] == 0) & (image_df['OCR_score'] >= 0.5)] 
print(len(fp))

In [ ]:
# Dataframe of false negatives

fn = image_df.loc[(image_dff['relevant'] == 1) & (image_df['OCR_score'] < 0.5)] 
print(len(fn))

In [ ]:
# Precision (What % of irdentified positives and true?)
precision = len(tp) / (len(tp) + len(fp))
print(precision)

In [ ]:
# PRecall (% of true positives identified)
recall = len(tp) / (len(tp) + len(fn))
print(recall)

In [ ]:
fn

In [41]:
image_df.to_csv('/home/studio-lab-user/DS-Projects/OCR/easy_ocr.csv')